In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# 경고문 없애기
import warnings
warnings.filterwarnings('ignore')

# 0. 데이터 불러오기

### (1) 랜딩클럽 데이터 불러오기

In [3]:
#데이터 불러오기
df = pd.read_csv("lending_club_2020_test.csv", index_col=0)

In [4]:
# 원본 보존
raw = df

In [5]:
# 초기화 버튼
#df = raw

In [6]:
# recoveries 열에서 결측치 갖는 행은 제외해버리기 (우리 전처리 계획 의거)
df = df.dropna(subset=['recoveries'])

In [7]:
df.isna().sum()

loan_amnt                                           0
funded_amnt                                         0
funded_amnt_inv                                     0
term                                                0
int_rate                                            0
                                               ...   
hardship_loan_status                          1119040
orig_projected_additional_accrued_interest    1104727
hardship_payoff_balance_amount                1103537
hardship_last_payment_amount                  1103537
debt_settlement_flag                                0
Length: 140, dtype: int64

### (2) 3년물, 5년물 국채 데이터 불러오기

#### 3년물

In [10]:
df_us3y = pd.read_csv('us03y.csv')

In [11]:
# '날짜' 열을 datetime 형식으로 변환
df_us3y['날짜'] = pd.to_datetime(df_us3y['날짜'])

# '날짜' 열을 연-월 기준으로 변환하여 새로운 '연월' 열 생성
df_us3y['연월'] = df_us3y['날짜'].dt.to_period('M').dt.to_timestamp()

# '연월' 기준으로 '종가'의 평균을 계산하여 새로운 데이터프레임 생성
df_monthly_avg_36 = df_us3y.groupby('연월')['종가'].mean().reset_index()

# 열 이름을 좀 더 명확하게 변경 (선택 사항)
df_monthly_avg_36.columns = ['연월', '월별 평균 종가']

#### 5년물

In [12]:
df_us5y = pd.read_csv('us05y.csv', encoding = 'EUC-KR')

In [13]:
# '날짜' 열을 datetime 형식으로 변환
df_us5y['날짜'] = pd.to_datetime(df_us5y['날짜'])

# '날짜' 열을 연-월 기준으로 변환하여 새로운 '연월' 열 생성
df_us5y['연월'] = df_us5y['날짜'].dt.to_period('M').dt.to_timestamp()

# '연월' 기준으로 '종가'의 평균을 계산하여 새로운 데이터프레임 생성
df_monthly_avg_60 = df_us5y.groupby('연월')['종가'].mean().reset_index()

# 열 이름을 좀 더 명확하게 변경 (선택 사항)
df_monthly_avg_60.columns = ['연월', '월별 평균 종가']

#### 랜딩클럽 데이터, 3년물 국채 데이터, 5년물 국채 데이터 호출 완료
df, df_monthly_avg_36, df_monthly_avg_60

# 1. `loan_status` 통합 작업

### 통합 기준
- **`Fully Paid`**: 
  - 원래 상태: `Fully Paid`
  - 정책 미충족 상태에서 완전히 상환된 경우: `Does not meet the credit policy. Status:Fully Paid`

- **`Charged Off`**:
  - 원래 상태: `Charged Off`
  - `Default`로 표시된 부도: `Default`
  - 정책 미충족 상태에서 부도 처리된 경우: `Does not meet the credit policy. Status:Charged Off`

- **`Current`**:
  - 31일 이상 연체된 대출: `Late (31-120 days)`
  - 16일에서 30일 사이 연체된 대출: `Late (16-30 days)`
  - 유예 기간에 있는 대출 (1~15일 연체): `In Grace Period`
  - 현재 정상 상태인 대출: `Current`

- **`Issued`**:
  - 발행된 대출: `Issued`


##### Late도 Current로 포함시킴

In [14]:
df_lending = df

In [15]:
df_lending['loan_status'].unique()

array(['Fully Paid', 'Charged Off',
       'Does not meet the credit policy. Status:Fully Paid',
       'Does not meet the credit policy. Status:Charged Off', 'Current',
       'Late (31-120 days)', 'In Grace Period', 'Late (16-30 days)',
       'Default', 'Issued'], dtype=object)

In [16]:
# loan_status 통합 작업
df_lending['loan_status'] = df_lending['loan_status'].replace({
    'Fully Paid': 'Fully Paid',
    'Does not meet the credit policy. Status:Fully Paid': 'Fully Paid',
    'Charged Off': 'Charged Off',
    'Default': 'Charged Off',
    'Does not meet the credit policy. Status:Charged Off': 'Charged Off',
    'Late (31-120 days)': 'Current',
    'Late (16-30 days)': 'Current',
    'In Grace Period': 'Current',
    'Current': 'Current',
    'Issued': 'Current'
})

In [17]:
df_lending['loan_status'].value_counts()

loan_status
Fully Paid     604205
Current        425933
Charged Off    146606
Name: count, dtype: int64

# 2. 결측치 처리

### (1) Column 1차 선별 (선정된 최종컬럼 + 파생변수 만드는데 필요한 녀석들)

1차때 선정한 변수에 'delinq_2yrs', 'pub_rec', 'collections_12_mths_ex_med' 이들을 추가했음

근데 pub_rec과 pub_rec_bankruptcies와 상관성이 너무 커 pub_rec 다시 제외

In [18]:
columns = [
    "loan_status", "funded_amnt", "term", "int_rate", "grade", "sub_grade", 
    "home_ownership", "annual_inc", "verification_status", 
    "purpose", "dti", "fico_range_low", "inq_last_6mths", 
    "total_pymnt", "application_type", "annual_inc_joint", 
    "dti_joint", "verification_status_joint", "tot_coll_amt", 
    "all_util", "chargeoff_within_12_mths", "delinq_amnt", 
    "num_actv_rev_tl", "pub_rec_bankruptcies", "tax_liens", 
    "sec_app_fico_range_low", 'emp_title', 
    'last_pymnt_d', 'issue_d', 'installment', 'addr_state', 
    'delinq_2yrs', 'collections_12_mths_ex_med',
    'loan_amnt'

    
]

df_filter = df_lending[columns]
df_filter

,loan_status,funded_amnt,term,int_rate,grade,sub_grade,home_ownership,annual_inc,verification_status,purpose,...,tax_liens,sec_app_fico_range_low,emp_title,last_pymnt_d,issue_d,installment,addr_state,delinq_2yrs,collections_12_mths_ex_med,loan_amnt
id,,,,,,,,,,,,,,,,,,,,,
1077175,Fully Paid,2400.0,36 months,15.96%,C,C5,RENT,12252.0,Not Verified,small_business,...,0.0,NaN,NaN,Jun-2014,Dec-2011,84.33,IL,0.0,0.0,2400.0
1076863,Fully Paid,10000.0,36 months,13.49%,C,C1,RENT,49200.0,Source Verified,other,...,0.0,NaN,AIR RESOURCES BOARD,Jan-2015,Dec-2011,339.31,CA,0.0,0.0,10000.0
1075269,Fully Paid,5000.0,36 months,7.90%,A,A4,RENT,36000.0,Source Verified,wedding,...,0.0,NaN,Veolia Transportaton,Jan-2015,Dec-2011,156.46,AZ,0.0,0.0,5000.0
1072053,Fully Paid,3000.0,36 months,18.64%,E,E1,RENT,48000.0,Source Verified,car,...,0.0,NaN,MKC Accounting,Jan-2015,Dec-2011,109.43,CA,0.0,0.0,3000.0
1071795,Charged Off,5600.0,60 months,21.28%,F,F2,OWN,40000.0,Source Verified,small_business,...,0.0,NaN,NaN,Apr-2012,Dec-2011,152.39,CA,0.0,0.0,5600.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101412243,Charged Off,3675.0,36 months,7.99%,A,A5,RENT,45000.0,Not Verified,credit_card,...,0.0,NaN,GROUP LEADER,Aug-2018,Apr-2017,115.15,NY,0.0,0.0,3675.0
102654880,Charged Off,30000.0,60 months,22.74%,E,E1,OWN,75000.0,Source Verified,debt_consolidation,...,0.0,675.0,Director of Marketing,Sep-2018,Apr-2017,841.24,AZ,0.0,0.0,30000.0
102156471,Fully Paid,29400.0,60 months,13.99%,C,C3,MORTGAGE,32000.0,Source Verified,debt_consolidation,...,0.0,660.0,Sales Manager,Nov-2019,Apr-2017,683.94,FL,0.0,0.0,29400.0


### (2) 개인채무/공동채무 대출에 대해서 분리

In [19]:
# application type에 따라 개인채무 or 공동채무 분할
df1 = df_filter[df_filter['application_type'] == "Individual"] # 개인 채무
df2 = df_filter[df_filter['application_type'] == "Joint App"] # 공동 채무

### (3) 결측치 채우기 위한 함수 정의

In [20]:
# 평균값 계산 (개인 채무)
grade_all_util_mean = round(df1.groupby('grade')['all_util'].mean())  # all_util
subgrade_num_actv_rev_mean = round(df1.groupby('sub_grade')['num_actv_rev_tl'].mean())  # num_actv_rev_tl
joint_sec_app_fico_range_low_mean = round(df1.groupby('application_type')['sec_app_fico_range_low'].mean())  # sec_app_fico_range_low

##### round 붙인 이유
all_util, num_actv_rev_tl: 각각 계좌 개수를 지정하는 변수이기 때문에 소수점이 나오면 안됨

sec_app_fico_range_low: 신용점수에 소수점이 존재하지 않음

In [21]:
# 결측치를 평균값으로 채우는 함수
def fill_missing_all_util(row):
    if pd.isnull(row['all_util']):
        return grade_all_util_mean.get(row['grade'], row['all_util'])
    else:
        return row['all_util']

def fill_missing_num_actv_rev_tl(row):
    if pd.isnull(row['num_actv_rev_tl']):
        return subgrade_num_actv_rev_mean.get(row['sub_grade'], row['num_actv_rev_tl'])
    else:
        return row['num_actv_rev_tl']

def fill_missing_sec_app_fico_range_low(row):
    if pd.isnull(row['sec_app_fico_range_low']):
        return joint_sec_app_fico_range_low_mean.get(row['application_type'], row['sec_app_fico_range_low'])
    else:
        return row['sec_app_fico_range_low']

### (4) 결측치 처리 시작

In [22]:
# 개인 채무 결측치 처리
df1.isnull().sum() #결측치 확인

df1=df1.dropna(how='all', axis=1) # OOO_joint, sec_app_OOO
df1.emp_title.fillna('inoccupation', inplace=True) # emp_title
df1=df1.dropna(subset=['dti']) # dti
df1=df1.dropna(subset=['fico_range_low']) # fico_range_low
df1=df1.dropna(subset=['total_pymnt']) # total_pymnt
df1=df1.dropna(subset=['annual_inc']) # annual_inc
df1.tot_coll_amt.fillna('0', inplace=True) # tot_coll_amt
df1.inq_last_6mths.fillna('0', inplace=True) # inq_last_6mths
df1.all_util=df1.apply(fill_missing_all_util, axis=1) # all_util
df1=df1.dropna(subset=['chargeoff_within_12_mths']) # chargeoff_within_12_mths
df1=df1.dropna(subset=['delinq_amnt']) # delinq_amnt
df1.num_actv_rev_tl=df1.apply(fill_missing_num_actv_rev_tl, axis=1) # num_actv_rev_tl
df1.pub_rec_bankruptcies.fillna('0', inplace=True) # pub_rec_bankruptcies
df1.tax_liens.fillna('0', inplace=True) #tax_liens
df1['last_pymnt_d'] = df1['last_pymnt_d'].fillna(df1['issue_d']) #issue_d

df1.isnull().sum()

loan_status                   0
funded_amnt                   0
term                          0
int_rate                      0
grade                         0
sub_grade                     0
home_ownership                0
annual_inc                    0
verification_status           0
purpose                       0
dti                           0
fico_range_low                0
inq_last_6mths                0
total_pymnt                   0
application_type              0
tot_coll_amt                  0
all_util                      0
chargeoff_within_12_mths      0
delinq_amnt                   0
num_actv_rev_tl               0
pub_rec_bankruptcies          0
tax_liens                     0
emp_title                     0
last_pymnt_d                  0
issue_d                       0
installment                   0
addr_state                    0
delinq_2yrs                   0
collections_12_mths_ex_med    0
loan_amnt                     0
dtype: int64

In [23]:
# # 2차 추가 변수
# df1['delinq_2yrs'] = df1['delinq_2yrs'].fillna(0)
# df1['pub_rec'] = df1['pub_rec'].fillna(0)
# df1['collections_12_mths_ex_med'] = df1['collections_12_mths_ex_med'].fillna(0)

##### 2차 추가 변수들 결측치는 개인대출에서는 이미 존재하지 않았음.

In [24]:
sum(df1.isna().sum())

0

# 3. 파생변수 제작

### (1) 코로나19 시기 대출 여부 (covid_impact)

In [25]:
# 'issue_d'를 datetime 형식으로 변환
df1['issue_d'] = pd.to_datetime(df1['issue_d'], format='%b-%Y')

# 'term'에서 개월 수를 추출하여 정수형으로 변환
df1['term_months'] = df1['term'].apply(lambda x: int(x.split()[0]))

# 대출 종료 날짜 계산
df1['loan_end_date'] = df1.apply(lambda row: row['issue_d'] + pd.DateOffset(months=row['term_months']), axis=1)


In [26]:
# 코로나19 영향을 받은 시기를 라벨링
covid_start_date = pd.to_datetime('2020-03-01')
df1['covid_impact'] = df1['loan_end_date'].apply(lambda x: 1 if x >= covid_start_date else 0)


### (2) 지역 변수 (region)

In [27]:
df1['addr_state'].unique()

array(['IL', 'CA', 'AZ', 'MO', 'TX', 'FL', 'NY', 'NJ', 'KY', 'OH', 'SC',
       'RI', 'CT', 'MA', 'OR', 'WI', 'MN', 'GA', 'PA', 'WA', 'NV', 'AK',
       'CO', 'VA', 'MD', 'WV', 'VT', 'DC', 'SD', 'NH', 'NC', 'NM', 'KS',
       'MI', 'MT', 'LA', 'AR', 'UT', 'AL', 'OK', 'WY', 'HI', 'DE', 'MS',
       'TN', 'NE', 'ID', 'IA', 'IN', 'ME', 'ND'], dtype=object)

In [28]:
# Make a list with each of the regions by state.

west = ['CA', 'OR', 'UT','WA', 'CO', 'NV', 'AK', 'MT', 'HI', 'WY', 'ID']
south_west = ['AZ', 'TX', 'NM', 'OK']
south_east = ['GA', 'NC', 'VA', 'FL', 'KY', 'SC', 'LA', 'AL', 'WV', 'DC', 'AR', 'DE', 'MS', 'TN' ]
mid_west = ['IL', 'MO', 'MN', 'OH', 'WI', 'KS', 'MI', 'SD', 'IA', 'NE', 'IN', 'ND']
north_east = ['CT', 'NY', 'PA', 'NJ', 'RI','MA', 'MD', 'VT', 'NH', 'ME']


# region 열 초기화
df1['region'] = np.nan

# 함수 정의 (서부, 남서부, 남동부, 중서부, 북동부로 분류)
def finding_regions(state):
    if state in west:
        return 'West'
    elif state in south_west:
        return 'SouthWest'
    elif state in south_east:
        return 'SouthEast'
    elif state in mid_west:
        return 'MidWest'
    elif state in north_east:
        return 'NorthEast'
    


df1['region'] = df1['addr_state'].apply(finding_regions)

In [29]:
df1.drop(columns = 'addr_state', inplace = True)

### (3) IRR (irr)

In [30]:
# 'last_pymnt_d'와 'issue_d' 열을 datetime 형식으로 변환
df1['last_pymnt_d'] = pd.to_datetime(df1['last_pymnt_d'])
df1['issue_d'] = pd.to_datetime(df1['issue_d'])

In [31]:
# months_between 계산
df1['months_between'] = (df1['last_pymnt_d'].dt.year - df1['issue_d'].dt.year) * 12 + (df1['last_pymnt_d'].dt.month - df1['issue_d'].dt.month)

In [32]:
df1.dropna(subset=['months_between', 'installment'], inplace=True)
df1['months_between'] = df1['months_between'].apply(lambda x: x+1 if x == 0 else x)
df1

,loan_status,funded_amnt,term,int_rate,grade,sub_grade,home_ownership,annual_inc,verification_status,purpose,...,issue_d,installment,delinq_2yrs,collections_12_mths_ex_med,loan_amnt,term_months,loan_end_date,covid_impact,region,months_between
id,,,,,,,,,,,,,,,,,,,,,
1077175,Fully Paid,2400.0,36 months,15.96%,C,C5,RENT,12252.0,Not Verified,small_business,...,2011-12-01,84.33,0.0,0.0,2400.0,36,2014-12-01,0,MidWest,30
1076863,Fully Paid,10000.0,36 months,13.49%,C,C1,RENT,49200.0,Source Verified,other,...,2011-12-01,339.31,0.0,0.0,10000.0,36,2014-12-01,0,West,37
1075269,Fully Paid,5000.0,36 months,7.90%,A,A4,RENT,36000.0,Source Verified,wedding,...,2011-12-01,156.46,0.0,0.0,5000.0,36,2014-12-01,0,SouthWest,37
1072053,Fully Paid,3000.0,36 months,18.64%,E,E1,RENT,48000.0,Source Verified,car,...,2011-12-01,109.43,0.0,0.0,3000.0,36,2014-12-01,0,West,37
1071795,Charged Off,5600.0,60 months,21.28%,F,F2,OWN,40000.0,Source Verified,small_business,...,2011-12-01,152.39,0.0,0.0,5600.0,60,2016-12-01,0,West,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103473466,Current,26000.0,60 months,12.74%,C,C1,MORTGAGE,100000.0,Not Verified,major_purchase,...,2017-04-01,588.13,3.0,0.0,26000.0,60,2022-04-01,1,SouthEast,37
103468691,Fully Paid,19200.0,60 months,12.74%,C,C1,OWN,70000.0,Not Verified,debt_consolidation,...,2017-04-01,434.31,3.0,0.0,19200.0,60,2022-04-01,1,NorthEast,30
101412243,Charged Off,3675.0,36 months,7.99%,A,A5,RENT,45000.0,Not Verified,credit_card,...,2017-04-01,115.15,0.0,0.0,3675.0,36,2020-04-01,1,NorthEast,16


In [33]:
#pip install numpy-financial

In [34]:
import numpy_financial as npf

### IRR 도출하는 함수
##### 오래걸림주의.. 약 4분

In [35]:
def calculate_irr(row):
    if (row['loan_status'] == 'Charged Off') or (row['loan_status'] == 'Current'):
        months_between = int(row['months_between'])
        cash_flows = [-row['funded_amnt']] + [row['total_pymnt'] / months_between] * months_between
        irr = npf.irr(cash_flows)
        annual_irr_percentage = (1 + irr)**12 - 1
        return annual_irr_percentage * 100
    
    if row['loan_status'] == 'Fully Paid':
        duration = int(row['term_months'])
        cash_flows = [-row['funded_amnt']] + [row['installment']] * duration
        irr = npf.irr(cash_flows)
        annual_irr_percentage = (1+irr)**12 - 1
        return annual_irr_percentage * 100
        
df1['irr'] = df1.apply(calculate_irr, axis=1).round(2)

In [36]:
# 잠깐백업
df_backup = df1

In [37]:
# 복원
#df1 = df_backup

In [38]:
# irr 결측치를 -100으로 채우기
# 결측치인 애들은 하나도 안갚은 애들이라 풀손실 = -100%
df1['irr'].fillna(-100, inplace=True)

In [39]:
# 잘 되었는지 확인
df1['irr']

id
1077175      17.18
1076863      14.36
1075269       8.20
1072053      20.32
1071795     -99.99
             ...  
103473466   -10.41
103468691    13.51
101412243   -51.23
102426796    32.78
102628603    -5.16
Name: irr, Length: 1092191, dtype: float64

In [40]:
df1['irr'].isna().sum()

0

### (4) 국채수익률

In [41]:
df[['issue_d']].head(3)

,issue_d
id,
1077175,Dec-2011
1076863,Dec-2011
1075269,Dec-2011


In [42]:
df_monthly_avg_36.head(3)

,연월,월별 평균 종가
0,2007-01-01,4.858923
1,2007-02-01,4.759158
2,2007-03-01,4.508364


In [43]:
df_monthly_avg_60.head(3)

,연월,월별 평균 종가
0,2006-01-01,4.362364
1,2006-02-01,4.569368
2,2006-03-01,4.716652


In [44]:
# 'issue_d' 열을 연-월 기준으로 변환하여 새로운 '연월' 열 생성
df1['연월'] = pd.to_datetime(df1['issue_d']).dt.to_period('M').dt.to_timestamp()

In [45]:
df1['연월']

id
1077175     2011-12-01
1076863     2011-12-01
1075269     2011-12-01
1072053     2011-12-01
1071795     2011-12-01
               ...    
103473466   2017-04-01
103468691   2017-04-01
101412243   2017-04-01
102426796   2017-04-01
102628603   2017-04-01
Name: 연월, Length: 1092191, dtype: datetime64[ns]

In [46]:
df_irr_making = df1

In [47]:
df_irr_making['연월']

id
1077175     2011-12-01
1076863     2011-12-01
1075269     2011-12-01
1072053     2011-12-01
1071795     2011-12-01
               ...    
103473466   2017-04-01
103468691   2017-04-01
101412243   2017-04-01
102426796   2017-04-01
102628603   2017-04-01
Name: 연월, Length: 1092191, dtype: datetime64[ns]

In [48]:
df_irr_making

,loan_status,funded_amnt,term,int_rate,grade,sub_grade,home_ownership,annual_inc,verification_status,purpose,...,delinq_2yrs,collections_12_mths_ex_med,loan_amnt,term_months,loan_end_date,covid_impact,region,months_between,irr,연월
id,,,,,,,,,,,,,,,,,,,,,
1077175,Fully Paid,2400.0,36 months,15.96%,C,C5,RENT,12252.0,Not Verified,small_business,...,0.0,0.0,2400.0,36,2014-12-01,0,MidWest,30,17.18,2011-12-01
1076863,Fully Paid,10000.0,36 months,13.49%,C,C1,RENT,49200.0,Source Verified,other,...,0.0,0.0,10000.0,36,2014-12-01,0,West,37,14.36,2011-12-01
1075269,Fully Paid,5000.0,36 months,7.90%,A,A4,RENT,36000.0,Source Verified,wedding,...,0.0,0.0,5000.0,36,2014-12-01,0,SouthWest,37,8.20,2011-12-01
1072053,Fully Paid,3000.0,36 months,18.64%,E,E1,RENT,48000.0,Source Verified,car,...,0.0,0.0,3000.0,36,2014-12-01,0,West,37,20.32,2011-12-01
1071795,Charged Off,5600.0,60 months,21.28%,F,F2,OWN,40000.0,Source Verified,small_business,...,0.0,0.0,5600.0,60,2016-12-01,0,West,4,-99.99,2011-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
103473466,Current,26000.0,60 months,12.74%,C,C1,MORTGAGE,100000.0,Not Verified,major_purchase,...,3.0,0.0,26000.0,60,2022-04-01,1,SouthEast,37,-10.41,2017-04-01
103468691,Fully Paid,19200.0,60 months,12.74%,C,C1,OWN,70000.0,Not Verified,debt_consolidation,...,3.0,0.0,19200.0,60,2022-04-01,1,NorthEast,30,13.51,2017-04-01
101412243,Charged Off,3675.0,36 months,7.99%,A,A5,RENT,45000.0,Not Verified,credit_card,...,0.0,0.0,3675.0,36,2020-04-01,1,NorthEast,16,-51.23,2017-04-01


In [49]:
df_irr_making['term'].value_counts()

term
 36 months    781148
 60 months    311043
Name: count, dtype: int64

In [50]:
df_irr_making.head()

,loan_status,funded_amnt,term,int_rate,grade,sub_grade,home_ownership,annual_inc,verification_status,purpose,...,delinq_2yrs,collections_12_mths_ex_med,loan_amnt,term_months,loan_end_date,covid_impact,region,months_between,irr,연월
id,,,,,,,,,,,,,,,,,,,,,
1077175,Fully Paid,2400.0,36 months,15.96%,C,C5,RENT,12252.0,Not Verified,small_business,...,0.0,0.0,2400.0,36,2014-12-01,0,MidWest,30,17.18,2011-12-01
1076863,Fully Paid,10000.0,36 months,13.49%,C,C1,RENT,49200.0,Source Verified,other,...,0.0,0.0,10000.0,36,2014-12-01,0,West,37,14.36,2011-12-01
1075269,Fully Paid,5000.0,36 months,7.90%,A,A4,RENT,36000.0,Source Verified,wedding,...,0.0,0.0,5000.0,36,2014-12-01,0,SouthWest,37,8.20,2011-12-01
1072053,Fully Paid,3000.0,36 months,18.64%,E,E1,RENT,48000.0,Source Verified,car,...,0.0,0.0,3000.0,36,2014-12-01,0,West,37,20.32,2011-12-01
1071795,Charged Off,5600.0,60 months,21.28%,F,F2,OWN,40000.0,Source Verified,small_business,...,0.0,0.0,5600.0,60,2016-12-01,0,West,4,-99.99,2011-12-01


In [51]:
df_monthly_avg_36.rename(columns = {'연월': 'issue_d'}, inplace = True)
df_monthly_avg_60.rename(columns = {'연월': 'issue_d'}, inplace = True)

In [52]:
df_irr_making_36 = df_irr_making.loc[df_irr_making['term'] == ' 36 months'].merge(df_monthly_avg_36, on='issue_d', how='left')
df_irr_making_60 = df_irr_making.loc[df_irr_making['term'] == ' 60 months'].merge(df_monthly_avg_60, on='issue_d', how='left')

In [53]:
df_irr_making = pd.concat([df_irr_making_36, df_irr_making_60], ignore_index=True)
df_irr_making

,loan_status,funded_amnt,term,int_rate,grade,sub_grade,home_ownership,annual_inc,verification_status,purpose,...,collections_12_mths_ex_med,loan_amnt,term_months,loan_end_date,covid_impact,region,months_between,irr,연월,월별 평균 종가
0,Fully Paid,2400.0,36 months,15.96%,C,C5,RENT,12252.0,Not Verified,small_business,...,0.0,2400.0,36,2014-12-01,0,MidWest,30,17.18,2011-12-01,0.381286
1,Fully Paid,10000.0,36 months,13.49%,C,C1,RENT,49200.0,Source Verified,other,...,0.0,10000.0,36,2014-12-01,0,West,37,14.36,2011-12-01,0.381286
2,Fully Paid,5000.0,36 months,7.90%,A,A4,RENT,36000.0,Source Verified,wedding,...,0.0,5000.0,36,2014-12-01,0,SouthWest,37,8.20,2011-12-01,0.381286
3,Fully Paid,3000.0,36 months,18.64%,E,E1,RENT,48000.0,Source Verified,car,...,0.0,3000.0,36,2014-12-01,0,West,37,20.32,2011-12-01,0.381286
4,Fully Paid,3000.0,36 months,9.91%,B,B1,RENT,15000.0,Source Verified,credit_card,...,0.0,3000.0,36,2014-12-01,0,MidWest,37,10.38,2011-12-01,0.381286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092186,Fully Paid,25275.0,60 months,28.69%,F,F1,MORTGAGE,98000.0,Not Verified,debt_consolidation,...,0.0,25275.0,60,2022-04-01,1,NorthEast,4,32.78,2017-04-01,1.815250
1092187,Current,15000.0,60 months,16.99%,D,D1,RENT,82360.0,Verified,debt_consolidation,...,0.0,15000.0,60,2022-04-01,1,SouthWest,37,-5.17,2017-04-01,1.815250
1092188,Current,26000.0,60 months,12.74%,C,C1,MORTGAGE,100000.0,Not Verified,major_purchase,...,0.0,26000.0,60,2022-04-01,1,SouthEast,37,-10.41,2017-04-01,1.815250
1092189,Fully Paid,19200.0,60 months,12.74%,C,C1,OWN,70000.0,Not Verified,debt_consolidation,...,0.0,19200.0,60,2022-04-01,1,NorthEast,30,13.51,2017-04-01,1.815250


### (5) 국채수익률 대비 순수익 (target)
target = IRR - 국채수익

In [54]:
df_irr_making['target'] = df_irr_making['irr'] - df_irr_making['월별 평균 종가'] 

In [55]:
df_irr_making

,loan_status,funded_amnt,term,int_rate,grade,sub_grade,home_ownership,annual_inc,verification_status,purpose,...,loan_amnt,term_months,loan_end_date,covid_impact,region,months_between,irr,연월,월별 평균 종가,target
0,Fully Paid,2400.0,36 months,15.96%,C,C5,RENT,12252.0,Not Verified,small_business,...,2400.0,36,2014-12-01,0,MidWest,30,17.18,2011-12-01,0.381286,16.798714
1,Fully Paid,10000.0,36 months,13.49%,C,C1,RENT,49200.0,Source Verified,other,...,10000.0,36,2014-12-01,0,West,37,14.36,2011-12-01,0.381286,13.978714
2,Fully Paid,5000.0,36 months,7.90%,A,A4,RENT,36000.0,Source Verified,wedding,...,5000.0,36,2014-12-01,0,SouthWest,37,8.20,2011-12-01,0.381286,7.818714
3,Fully Paid,3000.0,36 months,18.64%,E,E1,RENT,48000.0,Source Verified,car,...,3000.0,36,2014-12-01,0,West,37,20.32,2011-12-01,0.381286,19.938714
4,Fully Paid,3000.0,36 months,9.91%,B,B1,RENT,15000.0,Source Verified,credit_card,...,3000.0,36,2014-12-01,0,MidWest,37,10.38,2011-12-01,0.381286,9.998714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092186,Fully Paid,25275.0,60 months,28.69%,F,F1,MORTGAGE,98000.0,Not Verified,debt_consolidation,...,25275.0,60,2022-04-01,1,NorthEast,4,32.78,2017-04-01,1.815250,30.964750
1092187,Current,15000.0,60 months,16.99%,D,D1,RENT,82360.0,Verified,debt_consolidation,...,15000.0,60,2022-04-01,1,SouthWest,37,-5.17,2017-04-01,1.815250,-6.985250
1092188,Current,26000.0,60 months,12.74%,C,C1,MORTGAGE,100000.0,Not Verified,major_purchase,...,26000.0,60,2022-04-01,1,SouthEast,37,-10.41,2017-04-01,1.815250,-12.225250
1092189,Fully Paid,19200.0,60 months,12.74%,C,C1,OWN,70000.0,Not Verified,debt_consolidation,...,19200.0,60,2022-04-01,1,NorthEast,30,13.51,2017-04-01,1.815250,11.694750


# 4. 인코딩

### 범주/연속형 열 파악

In [56]:
df1 = df_irr_making

In [57]:
categorical_cols = []
continuous_cols = []

for col in df1.columns:
    if df1[col].dtype == 'object':
        categorical_cols.append(col)
    elif pd.api.types.is_numeric_dtype(df1[col]):
        continuous_cols.append(col)

print("범주형 열:", categorical_cols)
print('\n')
print("연속형 열:", continuous_cols)


범주형 열: ['loan_status', 'term', 'int_rate', 'grade', 'sub_grade', 'home_ownership', 'verification_status', 'purpose', 'inq_last_6mths', 'application_type', 'tot_coll_amt', 'pub_rec_bankruptcies', 'emp_title', 'region']


연속형 열: ['funded_amnt', 'annual_inc', 'dti', 'fico_range_low', 'total_pymnt', 'all_util', 'chargeoff_within_12_mths', 'delinq_amnt', 'num_actv_rev_tl', 'tax_liens', 'installment', 'delinq_2yrs', 'collections_12_mths_ex_med', 'loan_amnt', 'term_months', 'covid_impact', 'months_between', 'irr', '월별 평균 종가', 'target']


### (1) emp_title
Target Encoding By IRR

In [58]:
title_counts = df1['emp_title'].value_counts()
mask = df1['emp_title'].map(title_counts) < 10
df1.loc[mask, 'emp_title'] = '기타'
print(df1['emp_title'].value_counts())

emp_title_mean_irr = df1.groupby('emp_title')['irr'].mean()
df1['emp_title'] = df1['emp_title'].map(emp_title_mean_irr)

emp_title
기타                             388872
inoccupation                    91412
Teacher                         17423
Manager                         16420
Owner                            9483
                                ...  
host                               10
Front desk clerk                   10
Sales and Service Associate        10
Job Developer                      10
Meter Tech                         10
Name: count, Length: 8462, dtype: int64


### (2) term
one-hot encoding

In [59]:
# term에 대해 One-Hot Encoding 적용 (term_60mon 생성)
df1 = pd.get_dummies(df1, columns=['term'], prefix='term', drop_first=True)

df1.columns

Index(['loan_status', 'funded_amnt', 'int_rate', 'grade', 'sub_grade',
       'home_ownership', 'annual_inc', 'verification_status', 'purpose', 'dti',
       'fico_range_low', 'inq_last_6mths', 'total_pymnt', 'application_type',
       'tot_coll_amt', 'all_util', 'chargeoff_within_12_mths', 'delinq_amnt',
       'num_actv_rev_tl', 'pub_rec_bankruptcies', 'tax_liens', 'emp_title',
       'last_pymnt_d', 'issue_d', 'installment', 'delinq_2yrs',
       'collections_12_mths_ex_med', 'loan_amnt', 'term_months',
       'loan_end_date', 'covid_impact', 'region', 'months_between', 'irr',
       '연월', '월별 평균 종가', 'target', 'term_ 60 months'],
      dtype='object')

In [60]:
#pip install scikit-learn

### (3) loan_status
label encoding

- current = 1
- charged off = 0
- fully paid = 2

In [61]:
from sklearn.preprocessing import LabelEncoder

# Label Encoding 적용 (대상 열 수정)
label_enc_cols = ['loan_status']  # emp_title은 열 목록에 없으므로 제거
for col in label_enc_cols:
    df1[col] = LabelEncoder().fit_transform(df1[col])


### (4) `grade`, `sub_grade`, `int_rate`, `home_ownership`, `verification_status`, `purpose`, 
### `application_type`, `region`

In [62]:
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder

# grade 컬럼에 대해 Ordinal Encoding 적용
ordinal_enc = OrdinalEncoder(categories=[['A', 'B', 'C', 'D', 'E', 'F', 'G']])
df1['grade'] = ordinal_enc.fit_transform(df1[['grade']])


# int_rate 열에서 공백과 '%' 기호를 제거한 후 float로 변환
df1['int_rate'] = df1['int_rate'].str.strip().str.rstrip('%').astype(float)

# One-Hot Encoding 적용 (대상 열 수정)
one_hot_enc_cols = ['home_ownership', 'verification_status', 'purpose', 'application_type', 'region']
df1 = pd.get_dummies(df1, columns=one_hot_enc_cols)

#### sub_grade

In [63]:
from patsy import dmatrix
import pandas as pd

# 예시 데이터프레임 생성 (실제 데이터프레임 사용)
# df1 = pd.DataFrame({'sub_grade': [...]})

# sub_grade 값을 수동으로 숫자로 매핑하는 딕셔너리 생성
sub_grade_mapping = {f'{grade}{i}': idx for idx, (grade, i) in enumerate((grade, i) for grade in 'ABCDEFG' for i in range(1, 6))}
df1['sub_grade_numeric'] = df1['sub_grade'].map(sub_grade_mapping)  # 매핑 적용

# 선형 스플라인 처리 (매듭을 5개 설정하여 6개의 구간 생성)
sub_grade_spline = dmatrix("bs(sub_grade_numeric, knots=(5, 10, 15, 20, 25), degree=1, include_intercept=False)", 
                           {"sub_grade_numeric": df1['sub_grade_numeric']}, 
                           return_type='dataframe')

# 생성된 스플라인 변수의 이름을 간단하게 수정
sub_grade_spline.columns = [f'spline_segment_{i+1}' for i in range(sub_grade_spline.shape[1])]

# 스플라인 결과를 원래 데이터프레임에 병합
df1 = pd.concat([df1, sub_grade_spline], axis=1)


In [64]:
df1

,loan_status,funded_amnt,int_rate,grade,sub_grade,annual_inc,dti,fico_range_low,inq_last_6mths,total_pymnt,...,region_SouthWest,region_West,sub_grade_numeric,spline_segment_1,spline_segment_2,spline_segment_3,spline_segment_4,spline_segment_5,spline_segment_6,spline_segment_7
0,2,2400.0,15.96,2.0,C5,12252.0,8.72,735.0,2.0,3005.666844,...,False,False,14,1.0,0.0,0.2,0.8,0.0,0.0,0.0
1,2,10000.0,13.49,2.0,C1,49200.0,20.00,690.0,1.0,12231.890000,...,False,True,10,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2,5000.0,7.90,0.0,A4,36000.0,11.20,730.0,3.0,5632.210000,...,True,False,3,1.0,0.6,0.0,0.0,0.0,0.0,0.0
3,2,3000.0,18.64,4.0,E1,48000.0,5.35,660.0,2.0,3939.135294,...,False,True,20,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2,3000.0,9.91,1.0,B1,15000.0,12.56,705.0,2.0,3480.269999,...,False,False,5,1.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092186,2,25275.0,28.69,5.0,F1,98000.0,26.50,665.0,0.0,27664.954786,...,False,False,25,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1092187,1,15000.0,16.99,3.0,D1,82360.0,30.68,660.0,1.0,13776.110000,...,True,False,15,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1092188,1,26000.0,12.74,2.0,C1,100000.0,25.48,670.0,1.0,21742.410000,...,False,False,10,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1092189,2,19200.0,12.74,2.0,C1,70000.0,25.10,670.0,0.0,24222.056009,...,False,False,10,1.0,0.0,1.0,0.0,0.0,0.0,0.0


In [65]:
# from patsy import dmatrix
# import pandas as pd

# # 예시 데이터프레임 생성 (실제 데이터프레임 사용)
# # df1 = pd.DataFrame({'sub_grade': [...]})

# # sub_grade 값을 Ordinal Encoding에서 스플라인 처리로 변환
# df1['sub_grade'] = ordinal_enc.fit_transform(df1[['sub_grade']])  # 기존 Ordinal Encoding 유지

# # 선형 스플라인 처리 (매듭을 5개 설정하여 6개의 구간 생성)
# # 매듭은 데이터의 범위에 따라 설정하며, 여기는 sub_grade 범위 내 적절히 5개로 설정
# spline = dmatrix("bs(sub_grade, knots=(5, 10, 15, 20, 25), degree=1, include_intercept=False)", 
#                  {"sub_grade": df1['sub_grade']}, 
#                  return_type='dataframe')

# # 스플라인 결과를 데이터프레임에 병합
# df1 = pd.concat([df1, spline], axis=1)


In [66]:
# # sub_grade 컬럼에 대해 Ordinal Encoding 적용
# sub_grade_order = [f'{grade}{i}' for grade in 'ABCDEFG' for i in range(1, 6)]
# ordinal_enc = OrdinalEncoder(categories=[sub_grade_order])
# df1['sub_grade'] = ordinal_enc.fit_transform(df1[['sub_grade']])


In [67]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092191 entries, 0 to 1092190
Data columns (total 70 columns):
 #   Column                               Non-Null Count    Dtype         
---  ------                               --------------    -----         
 0   loan_status                          1092191 non-null  int64         
 1   funded_amnt                          1092191 non-null  float64       
 2   int_rate                             1092191 non-null  float64       
 3   grade                                1092191 non-null  float64       
 4   sub_grade                            1092191 non-null  object        
 5   annual_inc                           1092191 non-null  float64       
 6   dti                                  1092191 non-null  float64       
 7   fico_range_low                       1092191 non-null  float64       
 8   inq_last_6mths                       1092191 non-null  object        
 9   total_pymnt                          1092191 non-null  fl

### (5) `inq_last_6mths`, `tot_coll_amt`, `pub_rec_bankruptcies`
object로 되어있어 float로 수정

In [68]:
df1['inq_last_6mths'] = df1['inq_last_6mths'].astype(float)

In [69]:
df1['tot_coll_amt'] = df1['tot_coll_amt'].astype(float)

In [70]:
df1['pub_rec_bankruptcies'] = df1['pub_rec_bankruptcies'].astype(float)

In [71]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092191 entries, 0 to 1092190
Data columns (total 70 columns):
 #   Column                               Non-Null Count    Dtype         
---  ------                               --------------    -----         
 0   loan_status                          1092191 non-null  int64         
 1   funded_amnt                          1092191 non-null  float64       
 2   int_rate                             1092191 non-null  float64       
 3   grade                                1092191 non-null  float64       
 4   sub_grade                            1092191 non-null  object        
 5   annual_inc                           1092191 non-null  float64       
 6   dti                                  1092191 non-null  float64       
 7   fico_range_low                       1092191 non-null  float64       
 8   inq_last_6mths                       1092191 non-null  float64       
 9   total_pymnt                          1092191 non-null  fl

In [72]:
sum(df1.isna().sum())

0

In [73]:
df1

,loan_status,funded_amnt,int_rate,grade,sub_grade,annual_inc,dti,fico_range_low,inq_last_6mths,total_pymnt,...,region_SouthWest,region_West,sub_grade_numeric,spline_segment_1,spline_segment_2,spline_segment_3,spline_segment_4,spline_segment_5,spline_segment_6,spline_segment_7
0,2,2400.0,15.96,2.0,C5,12252.0,8.72,735.0,2.0,3005.666844,...,False,False,14,1.0,0.0,0.2,0.8,0.0,0.0,0.0
1,2,10000.0,13.49,2.0,C1,49200.0,20.00,690.0,1.0,12231.890000,...,False,True,10,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2,5000.0,7.90,0.0,A4,36000.0,11.20,730.0,3.0,5632.210000,...,True,False,3,1.0,0.6,0.0,0.0,0.0,0.0,0.0
3,2,3000.0,18.64,4.0,E1,48000.0,5.35,660.0,2.0,3939.135294,...,False,True,20,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2,3000.0,9.91,1.0,B1,15000.0,12.56,705.0,2.0,3480.269999,...,False,False,5,1.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092186,2,25275.0,28.69,5.0,F1,98000.0,26.50,665.0,0.0,27664.954786,...,False,False,25,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1092187,1,15000.0,16.99,3.0,D1,82360.0,30.68,660.0,1.0,13776.110000,...,True,False,15,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1092188,1,26000.0,12.74,2.0,C1,100000.0,25.48,670.0,1.0,21742.410000,...,False,False,10,1.0,0.0,1.0,0.0,0.0,0.0,0.0
1092189,2,19200.0,12.74,2.0,C1,70000.0,25.10,670.0,0.0,24222.056009,...,False,False,10,1.0,0.0,1.0,0.0,0.0,0.0,0.0


##### Encoding 완료

# 6. 데이터 내보내기

### (1) Current와 Current 아닌 대출자 데이터 분리

참고. Current에 issued도 포함되어있음

#### 헷갈리지말자..

- loan_status
- Fully Paid     899745
- Current        637455
- Charged Off    218094
- Name: count, dtype: int64

In [74]:
df1['loan_status'].value_counts()

loan_status
2    582878
1    369451
0    139862
Name: count, dtype: int64

In [75]:
df1[df1['loan_status'] == 1]

,loan_status,funded_amnt,int_rate,grade,sub_grade,annual_inc,dti,fico_range_low,inq_last_6mths,total_pymnt,...,region_SouthWest,region_West,sub_grade_numeric,spline_segment_1,spline_segment_2,spline_segment_3,spline_segment_4,spline_segment_5,spline_segment_6,spline_segment_7
72581,1,16000.0,7.97,0.0,A5,79077.0,15.94,700.0,0.0,15527.57,...,False,False,4,1.0,0.8,0.0,0.0,0.0,0.000000,0.000000
72589,1,4000.0,9.44,1.0,B1,52000.0,33.81,690.0,0.0,4092.44,...,False,False,5,1.0,1.0,0.0,0.0,0.0,0.000000,0.000000
72594,1,21475.0,17.09,3.0,D1,53000.0,33.06,700.0,1.0,24490.74,...,True,False,15,1.0,0.0,0.0,1.0,0.0,0.000000,0.000000
72596,1,12000.0,10.42,1.0,B3,55000.0,8.49,680.0,0.0,12452.67,...,False,False,7,1.0,0.6,0.4,0.0,0.0,0.000000,0.000000
72599,1,8000.0,7.07,0.0,A2,81000.0,26.83,775.0,0.0,7669.85,...,False,True,1,1.0,0.2,0.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092179,1,33100.0,29.99,5.0,F3,175000.0,21.72,680.0,0.0,39560.75,...,False,False,27,1.0,0.0,0.0,0.0,0.0,0.777778,0.222222
1092182,1,24300.0,30.49,5.0,F4,74000.0,21.39,695.0,4.0,29420.48,...,False,False,28,1.0,0.0,0.0,0.0,0.0,0.666667,0.333333
1092183,1,13000.0,25.49,4.0,E4,50000.0,28.01,665.0,1.0,14238.43,...,False,True,23,1.0,0.0,0.0,0.0,0.4,0.600000,0.000000
1092187,1,15000.0,16.99,3.0,D1,82360.0,30.68,660.0,1.0,13776.11,...,True,False,15,1.0,0.0,0.0,1.0,0.0,0.000000,0.000000


In [76]:
# Current와 Issued 상태를 따로 보관
df_current = df1[df1['loan_status'] == 1].copy()

# df1에서 Current와 Issued 상태 제거
df_not_current = df1[df1['loan_status'] != 1]


In [77]:
df_current['loan_status'].value_counts()

loan_status
1    369451
Name: count, dtype: int64

### (2) Current들 중에서 irr이 양수인 경우만 합치기 (근데 회귀에선 제외해야할수도? 그래서 분리)

In [78]:
df_target_pos = df_current[df_current['target'] >= 0]

In [79]:
df_target_pos

,loan_status,funded_amnt,int_rate,grade,sub_grade,annual_inc,dti,fico_range_low,inq_last_6mths,total_pymnt,...,region_SouthWest,region_West,sub_grade_numeric,spline_segment_1,spline_segment_2,spline_segment_3,spline_segment_4,spline_segment_5,spline_segment_6,spline_segment_7
72589,1,4000.0,9.44,1.0,B1,52000.0,33.81,690.0,0.0,4092.44,...,False,False,5,1.0,1.0,0.0,0.0,0.0,0.000000,0.000000
72594,1,21475.0,17.09,3.0,D1,53000.0,33.06,700.0,1.0,24490.74,...,True,False,15,1.0,0.0,0.0,1.0,0.0,0.000000,0.000000
72596,1,12000.0,10.42,1.0,B3,55000.0,8.49,680.0,0.0,12452.67,...,False,False,7,1.0,0.6,0.4,0.0,0.0,0.000000,0.000000
72600,1,1200.0,23.88,4.0,E2,30000.0,18.32,670.0,1.0,1501.14,...,False,False,21,1.0,0.0,0.0,0.0,0.8,0.200000,0.000000
72601,1,2500.0,12.62,2.0,C1,44064.0,12.20,660.0,0.0,2677.45,...,False,True,10,1.0,0.0,1.0,0.0,0.0,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092158,1,20000.0,30.49,5.0,F4,85000.0,24.04,670.0,0.0,24131.19,...,False,False,28,1.0,0.0,0.0,0.0,0.0,0.666667,0.333333
1092170,1,11000.0,22.74,4.0,E1,67784.0,10.68,670.0,1.0,11399.12,...,False,False,20,1.0,0.0,0.0,0.0,1.0,0.000000,0.000000
1092179,1,33100.0,29.99,5.0,F3,175000.0,21.72,680.0,0.0,39560.75,...,False,False,27,1.0,0.0,0.0,0.0,0.0,0.777778,0.222222
1092182,1,24300.0,30.49,5.0,F4,74000.0,21.39,695.0,4.0,29420.48,...,False,False,28,1.0,0.0,0.0,0.0,0.0,0.666667,0.333333


In [80]:
df_with_pos_current = pd.concat([df_not_current, df_target_pos], ignore_index=True)

#### 만들어진 데이터프레임 2개: df_with_pos_current, df_not_current

In [81]:
df_with_pos_current.shape

(758318, 70)

In [82]:
df_not_current.shape

(722740, 70)

In [83]:
df_not_current

,loan_status,funded_amnt,int_rate,grade,sub_grade,annual_inc,dti,fico_range_low,inq_last_6mths,total_pymnt,...,region_SouthWest,region_West,sub_grade_numeric,spline_segment_1,spline_segment_2,spline_segment_3,spline_segment_4,spline_segment_5,spline_segment_6,spline_segment_7
0,2,2400.0,15.96,2.0,C5,12252.0,8.72,735.0,2.0,3005.666844,...,False,False,14,1.0,0.0,0.2,0.8,0.0,0.0,0.0
1,2,10000.0,13.49,2.0,C1,49200.0,20.00,690.0,1.0,12231.890000,...,False,True,10,1.0,0.0,1.0,0.0,0.0,0.0,0.0
2,2,5000.0,7.90,0.0,A4,36000.0,11.20,730.0,3.0,5632.210000,...,True,False,3,1.0,0.6,0.0,0.0,0.0,0.0,0.0
3,2,3000.0,18.64,4.0,E1,48000.0,5.35,660.0,2.0,3939.135294,...,False,True,20,1.0,0.0,0.0,0.0,1.0,0.0,0.0
4,2,3000.0,9.91,1.0,B1,15000.0,12.56,705.0,2.0,3480.269999,...,False,False,5,1.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092184,2,25000.0,16.99,3.0,D1,141000.0,21.23,675.0,0.0,32971.833202,...,False,False,15,1.0,0.0,0.0,1.0,0.0,0.0,0.0
1092185,2,24500.0,22.74,4.0,E1,59716.0,30.65,675.0,1.0,30405.493454,...,False,False,20,1.0,0.0,0.0,0.0,1.0,0.0,0.0
1092186,2,25275.0,28.69,5.0,F1,98000.0,26.50,665.0,0.0,27664.954786,...,False,False,25,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1092189,2,19200.0,12.74,2.0,C1,70000.0,25.10,670.0,0.0,24222.056009,...,False,False,10,1.0,0.0,1.0,0.0,0.0,0.0,0.0


### (2) 최종컬럼만 남기기

## 최종 컬럼
- num_actv_rev_tl
- dti
- fico_range_low
- funded_amnt
- term
- int_rate
- grade / sub_grade
- emp_title
- home_ownership
- annual_inc
- verification_status
- purpose
- pub_rec_bankruptcies
- tax_liens
- all_util
- chargeoff_within_12_mths
- tot_coll_amt
- application_type
- annual_inc_joint
- dti_joint
- verification_status_joint
- covid_impact
- inq_last_6mths
- irr

## 제외한 컬럼
- application_type_Individual


In [84]:
final_columns = [
    'num_actv_rev_tl', 'dti', 'fico_range_low', 'funded_amnt', 'int_rate', 
    'grade', 'emp_title', 'annual_inc', 
    'verification_status_Not Verified', 'verification_status_Source Verified', 'verification_status_Verified',
    'purpose_car', 'purpose_credit_card', 'purpose_debt_consolidation',
    'purpose_educational', 'purpose_home_improvement', 'purpose_house',
    'purpose_major_purchase', 'purpose_medical', 'purpose_moving',
    'purpose_other', 'purpose_renewable_energy', 'purpose_small_business',
    'purpose_vacation', 'purpose_wedding',
    'pub_rec_bankruptcies', 'tax_liens', 
    'all_util', 'chargeoff_within_12_mths', 'tot_coll_amt', 
    'covid_impact', 'inq_last_6mths', 'delinq_amnt', 'total_pymnt',
    'term_ 60 months', 
    'home_ownership_ANY', 'home_ownership_MORTGAGE', 'home_ownership_NONE', 'home_ownership_OTHER',
    'home_ownership_OWN', 'home_ownership_RENT', 
    'delinq_2yrs', 'collections_12_mths_ex_med',
    "spline_segment_1", "spline_segment_2", "spline_segment_3", "spline_segment_4", "spline_segment_5", "spline_segment_6", "spline_segment_7",
    'irr', 'target'
]


In [85]:
df_final_1 = df_not_current[final_columns]

In [86]:
df_final_2 = df_with_pos_current[final_columns]

In [87]:
df_final_1.isna().sum()

num_actv_rev_tl                        0
dti                                    0
fico_range_low                         0
funded_amnt                            0
int_rate                               0
grade                                  0
emp_title                              0
annual_inc                             0
verification_status_Not Verified       0
verification_status_Source Verified    0
verification_status_Verified           0
purpose_car                            0
purpose_credit_card                    0
purpose_debt_consolidation             0
purpose_educational                    0
purpose_home_improvement               0
purpose_house                          0
purpose_major_purchase                 0
purpose_medical                        0
purpose_moving                         0
purpose_other                          0
purpose_renewable_energy               0
purpose_small_business                 0
purpose_vacation                       0
purpose_wedding 

In [88]:
df_final_2.isna().sum()

num_actv_rev_tl                        0
dti                                    0
fico_range_low                         0
funded_amnt                            0
int_rate                               0
grade                                  0
emp_title                              0
annual_inc                             0
verification_status_Not Verified       0
verification_status_Source Verified    0
verification_status_Verified           0
purpose_car                            0
purpose_credit_card                    0
purpose_debt_consolidation             0
purpose_educational                    0
purpose_home_improvement               0
purpose_house                          0
purpose_major_purchase                 0
purpose_medical                        0
purpose_moving                         0
purpose_other                          0
purpose_renewable_energy               0
purpose_small_business                 0
purpose_vacation                       0
purpose_wedding 

### (3) CSV Export

In [89]:
df_final_1.to_csv('전처리_3차_without_current.csv', index = False) # current중 흑자인거 포함
df_final_2.to_csv('전처리_3차_with_pos_current.csv', index = False) # current 싹다 제거

In [90]:
df_final_1.shape

(722740, 52)

In [91]:
df_final_2.shape

(758318, 52)